In [4]:
pip install requests psycopg2-binary schedule python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import csv  
    

In [4]:
import requests
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime

# =============================
# LOAD ENV VARIABLES
# =============================
load_dotenv()

API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

if not API_KEY:
    raise ValueError("ALPHA_VANTAGE_API_KEY not found in .env file")

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "database": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "port": os.getenv("DB_PORT")
}

SYMBOL = "ENGI.PA"   # Change symbol here

# =============================
# DB CONNECTION
# =============================
def get_connection():
    return psycopg2.connect(**DB_CONFIG)

# =============================
# FETCH LAST 60 DAYS
# =============================
def fetch_last_60_days(symbol):
    try:
        url = (
            "https://www.alphavantage.co/query"
            f"?function=TIME_SERIES_DAILY"
            f"&symbol={symbol}"
            f"&outputsize=compact"
            f"&apikey={API_KEY}"
        )

        response = requests.get(url, timeout=10)
        data = response.json()

        if "Time Series (Daily)" not in data:
            print("API Response:", data)
            return None

        series = data["Time Series (Daily)"]

        # Get latest 60 dates
        sorted_dates = sorted(series.keys(), reverse=True)[:60]

        candles = []

        for date_str in sorted_dates:
            daily = series[date_str]

            candles.append({
                "symbol": symbol,
                "open": float(daily["1. open"]),
                "high": float(daily["2. high"]),
                "low": float(daily["3. low"]),
                "close": float(daily["4. close"]),
                "volume": int(daily["5. volume"]),
                "timestamp": datetime.strptime(date_str, "%Y-%m-%d").date()
            })

        return candles

    except Exception as e:
        print("Fetch error:", e)
        return None

# =============================
# SAVE MULTIPLE ROWS
# =============================
def save_multiple_to_db(candles):
    try:
        conn = get_connection()
        cur = conn.cursor()

        insert_query = """
            INSERT INTO stock_prices_daily
            (symbol, open, high, low, close, volume, timestamp)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (symbol, timestamp) DO UPDATE SET
                open = EXCLUDED.open,
                high = EXCLUDED.high,
                low = EXCLUDED.low,
                close = EXCLUDED.close,
                volume = EXCLUDED.volume;
        """

        for candle in candles:
            cur.execute(insert_query, (
                candle["symbol"],
                candle["open"],
                candle["high"],
                candle["low"],
                candle["close"],
                candle["volume"],
                candle["timestamp"]
            ))

        conn.commit()
        cur.close()
        conn.close()

        print(f"Inserted/Updated {len(candles)} records")

    except Exception as e:
        print("Database insert error:", e)

# =============================
# MAIN
# =============================
def main():
    print(f"Fetching last 60 days for {SYMBOL}...\n")

    candles = fetch_last_60_days(SYMBOL)

    if candles:
        save_multiple_to_db(candles)
        print("Done.")
    else:
        print("No data fetched.")

if __name__ == "__main__":
    main()


Fetching last 60 days for ENGI.PA...

Inserted/Updated 60 records
Done.


In [ ]:
import requests
import psycopg2
import os
import time
import numpy as np
import joblib
from dotenv import load_dotenv
from datetime import datetime
from tensorflow.keras.models import load_model

# ==========================================
# LOAD ENVIRONMENT VARIABLES
# ==========================================
load_dotenv()

API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")

if not API_KEY:
    raise ValueError("ALPHA_VANTAGE_API_KEY not found in .env file")

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "database": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "port": os.getenv("DB_PORT")
}

STOCKS = ["ENGI.PA"]
N_PAST = 60

# ==========================================
# LOAD MODEL + SCALER
# ==========================================
print("Loading LSTM model...")
model = load_model("lstm_model.h5", compile=False)

print("Loading scaler...")
scaler = joblib.load("scaler.pkl")

# ==========================================
# DATABASE CONNECTION
# ==========================================
def get_connection():
    return psycopg2.connect(**DB_CONFIG)

# ==========================================
# FETCH LATEST DAILY CANDLE
# ==========================================
def fetch_latest_candle(symbol):
    try:
        url = (
            "https://www.alphavantage.co/query"
            f"?function=TIME_SERIES_DAILY"
            f"&symbol={symbol}"
            f"&outputsize=compact"
            f"&apikey={API_KEY}"
        )

        response = requests.get(url, timeout=10)
        data = response.json()

        if "Time Series (Daily)" not in data:
            print("API Response:", data)
            return None

        series = data["Time Series (Daily)"]

        # Get most recent trading day
        latest_date = sorted(series.keys(), reverse=True)[0]
        latest = series[latest_date]

        return {
            "symbol": symbol,
            "open": float(latest["1. open"]),
            "high": float(latest["2. high"]),
            "low": float(latest["3. low"]),
            "close": float(latest["4. close"]),
            "volume": int(latest["5. volume"]),
            "timestamp": datetime.strptime(latest_date, "%Y-%m-%d").date()
        }

    except Exception as e:
        print("Fetch error:", e)
        return None

# ==========================================
# SAVE CANDLE TO DATABASE
# ==========================================
def save_to_db(data):
    try:
        conn = get_connection()
        cur = conn.cursor()

        insert_query = """
            INSERT INTO stock_prices_daily
            (symbol, open, high, low, close, volume, timestamp)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (symbol, timestamp) DO NOTHING;
        """

        cur.execute(insert_query, (
            data["symbol"],
            data["open"],
            data["high"],
            data["low"],
            data["close"],
            data["volume"],
            data["timestamp"]
        ))

        conn.commit()
        cur.close()
        conn.close()

        print(f"Saved: {data['symbol']} at {data['timestamp']}")

    except Exception as e:
        print("Database insert error:", e)

# ==========================================
# FETCH LAST 60 DAYS FROM DB
# ==========================================
def get_last_n_days(symbol, n=N_PAST):
    try:
        conn = get_connection()
        cur = conn.cursor()

        query = """
            SELECT open, high, low, close
            FROM stock_prices_daily
            WHERE symbol = %s
            ORDER BY timestamp DESC
            LIMIT %s;
        """

        cur.execute(query, (symbol, n))
        rows = cur.fetchall()

        cur.close()
        conn.close()

        if len(rows) < n:
            print("Not enough historical data for prediction")
            return None

        rows.reverse()

        import pandas as pd
        df = pd.DataFrame(rows, columns=['open','high','low','close'])

        return df

    except Exception as e:
        print("DB fetch error:", e)
        return None
# ==========================================
# PREDICT NEXT DAY CLOSE
# ==========================================
def predict_next_day(symbol):
    data = get_last_n_days(symbol)

    if data is None:
        return

    # Scale
    scaled_data = scaler.transform(data)

    # Reshape for LSTM
    X_input = scaled_data.reshape(1, N_PAST, 4)

    # Predict
    prediction_scaled = model.predict(X_input, verbose=0)

    # Inverse scale (close is index 3)
    dummy = np.zeros((1, 4))
    dummy[0, 3] = prediction_scaled[0, 0]

    predicted_price = scaler.inverse_transform(dummy)[0, 3]

    print(f"\nPredicted NEXT DAY close for {symbol}: {predicted_price:.2f}\n")

    return predicted_price
    

# ==========================================
# MAIN LOOP
# ==========================================
def main():
    print("Starting DAILY ingestion + LSTM prediction...\n")

    while True:
        for symbol in STOCKS:
            data = fetch_latest_candle(symbol)

            if data:
                save_to_db(data)
                predict_next_day(symbol)

        print("Sleeping 24 hours...\n")
        time.sleep(86400)

if __name__ == "__main__":
    main()

Loading LSTM model...
Loading scaler...
Starting DAILY ingestion + LSTM prediction...

Saved: ENGI.PA at 2026-02-18

Predicted NEXT DAY close for ENGI.PA: 26.56

Sleeping 24 hours...

